# Descripción del problema

Los datos provienen de una planta minera.
El objetivo principal de estos datos es poder predecir la impureza en el concentrado de mineral extraído. Esta impureza es medida cada hora. Con las predicciones podríamos ayudar a los ingenieros con una alerta temprana para que puedan tomar acciones correctivas. 


- `% Silica Concentrate` es la concentración de sílice (laboratorio).
- `% Iron Concentrate` es la concentración de hierro (laboratorio).
- El concentrado de sílice es la impureza en el mineral de hierro que debe eliminarse.
- El proceso actual de detección de sílice lleva muchas horas.


Con la ayuda de algunos análisis y modelos de datos, podemos dar una buena aproximación del concentrado de sílice que reducirá mucho tiempo y esfuerzos necesarios para procesar el mineral de hierro


Fuente:
https://www.kaggle.com/edumagalhaes/quality-prediction-in-a-mining-process


## Librerias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

%matplotlib inline

## Lectura de datos

In [ ]:
df = pd.read_csv('data/MiningProcess_Flotation_Plant_Database.csv', decimal=",").drop_duplicates()

In [ ]:
df = df.drop(["date","% Iron Concentrate"],axis = 1)

In [ ]:
df = df.sample(10000)

In [ ]:
df.head()

In [ ]:
df.describe()

## Preparación de datos

In [ ]:
y = df['% Silica Concentrate']
X = df.drop(['% Silica Concentrate'], axis=1)

In [ ]:
scaler_X = preprocessing.MinMaxScaler()
scaler_y = preprocessing.MinMaxScaler()

X = pd.DataFrame(scaler_X.fit_transform(X),columns=X.columns)
y = scaler_y.fit_transform(y.values.reshape(-1, 1))


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=103)

In [ ]:
def plot(y_train,y_train_hat,y_test,y_test_hat):
    #Definimos los limites de la  grafica
    plt.figure(figsize=(10,10))
    plt.axis([0,1,0,1])
    plt.plot([[0,0],[1,1]], "r--")
    #Grafica de dispersión (Real vs predicción)
    plt.scatter(y_train,y_train_hat, label="Entrenamiento",alpha=0.8)
    plt.scatter(y_test,y_test_hat, label="Prueba", alpha=0.8)
    #Preparamos las etiquetas
    plt.legend()
    plt.xlabel("Y Real")
    plt.ylabel("Y Predicción")
    #Calculamos error cuadrático medio
    error = mean_squared_error(scaler_y.inverse_transform(y_test.reshape(-1, 1)),
                               scaler_y.inverse_transform(y_test_hat.reshape(-1, 1)))
    print("El error cuadrático medio es {}".format(error))

## Regresión Lineal

In [ ]:
from sklearn.linear_model import LinearRegression
#Creando el modelo y entrenando
reg = LinearRegression().fit(X_train,y_train)
#Prediciendo valores de entrenamiento
y_train_hat = reg.predict(X_train)
#Prediciendo valores de validación
y_test_hat = reg.predict(X_test)

In [ ]:
plot(y_train,y_train_hat,y_test,y_test_hat)

## SVR

In [ ]:
from sklearn.svm import SVR

svr_rbf = SVR(kernel='rbf', gamma='auto')
svr_rbf.fit(X_train,y_train.ravel())
#Prediciendo valores de entrenamiento
y_train_hat = svr_rbf.predict(X_train)
#Prediciendo valores de validación
y_test_hat = svr_rbf.predict(X_test)

plot(y_train,y_train_hat,y_test,y_test_hat)


In [ ]:
svr_poly = SVR(kernel='poly', gamma='auto')
svr_poly.fit(X_train,y_train.ravel())
#Prediciendo valores de entrenamiento
y_train_hat = svr_poly.predict(X_train)
#Prediciendo valores de validación
y_test_hat = svr_poly.predict(X_test)

plot(y_train,y_train_hat,y_test,y_test_hat)

## Arbol de decisión

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=12,min_samples_leaf=50)
tree.fit(X_train,y_train.ravel())
#Prediciendo valores de entrenamiento
y_train_hat = tree.predict(X_train)
#Prediciendo valores de validación
y_test_hat = tree.predict(X_test)

plot(y_train,y_train_hat,y_test,y_test_hat)



# Selección de variables

In [ ]:
from sklearn.feature_selection import RFE

estimator = LinearRegression()
selector = RFE(estimator, 8, step=1)
selector = selector.fit(X_train, y_train.ravel())

In [ ]:
X_train_reduced = X_train.loc[:, selector.support_]
X_test_reduced = X_test.loc[:, selector.support_]
X_train_reduced.head()

In [ ]:
svr_rbf = SVR(kernel='rbf', gamma='auto')
svr_rbf.fit(X_train_reduced,y_train.ravel())
#Prediciendo valores de entrenamiento
y_train_hat = svr_rbf.predict(X_train_reduced)
#Prediciendo valores de validación
y_test_hat = svr_rbf.predict(X_test_reduced)

plot(y_train,y_train_hat,y_test,y_test_hat)


# Métodos de ensamble

Los métodos de ensamble de modelos utilizan múltiples algoritmos de aprendizaje para obtener un mejor rendimiento predictivo que el que se podría obtener de cualquiera de los algoritmos de aprendizaje de manera individual.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train,y_train.ravel())

#Prediciendo valores de entrenamiento
y_train_hat = rf.predict(X_train)
#Prediciendo valores de validación
y_test_hat = rf.predict(X_test)

plot(y_train,y_train_hat,y_test,y_test_hat)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=40,  max_depth=12, learning_rate=0.1)
gbr.fit(X_train,y_train.ravel())

#Prediciendo valores de entrenamiento
y_train_hat = gbr.predict(X_train)
#Prediciendo valores de validación
y_test_hat = gbr.predict(X_test)

plot(y_train,y_train_hat,y_test,y_test_hat)